In [129]:
pip install imgkit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import os
import pandas as pd
import dataframe_image as dfi

pd.set_option('display.max_rows',None, 'display.max_columns', None)

# CAPTURE UNPIVOT TABLE

In [149]:
df_quadran = pd.read_csv('../quadran/target.csv')
#target_pivot = df_quadran[['KABUPATEN/KOTA','Jumlah_DPT_2024','DPT_x_82%','TARGET_GP_2024','% TARGET_GP_2024']]
df_quadran.head()

,PROPINSI,KABUPATEN2,Kecamatan,Kelurahan,Jumlah TPS,Jumlah DPT 2024,DPT 2024 x 82%,Target GP 2024,% TARGET GP 2024,Kuadran,Prio
0,ACEH,ACEH BARAT,WOYLA BARAT,ULEE PULO,2.0,337,276,11,4.00%,4,0
1,ACEH,ACEH BARAT,BUBON,ULEE BLANG,2.0,426,349,36,10.30%,4,0
2,ACEH,ACEH BARAT,MEUREUBO,UJONG TANJONG,1.0,251,206,34,16.70%,4,0
3,ACEH,ACEH BARAT,JOHAN PAHLAWAN,UJONG BAROH,2.0,255,209,120,57.30%,3,0
4,ACEH,ACEH BARAT,WOYLA TIMUR,TUWI EUMPEUK,5.0,1236,1014,407,40.20%,4,0


In [150]:
colum = df_quadran.columns[4:10].to_list()
colum

['Jumlah TPS',
 'Jumlah DPT 2024',
 'DPT 2024 x 82%',
 'Target GP 2024',
 '% TARGET GP 2024',
 'Kuadran']

In [151]:
provinsi = df_quadran['PROPINSI'].unique().tolist()
kabkot = df_quadran['KABUPATEN2'].unique().tolist()
kecamatan = df_quadran['Kecamatan'].unique().tolist()
provinsi

['ACEH',
 'SUMATERA BARAT',
 'NUSA TENGGARA TIMUR',
 'SUMATERA UTARA',
 'PAPUA',
 'BALI',
 'KALIMANTAN SELATAN',
 'JAWA BARAT',
 'SULAWESI TENGAH',
 'KEPULAUAN BANGKA BELITUNG',
 'JAWA TIMUR',
 'JAWA TENGAH',
 'SULAWESI SELATAN',
 'DAERAH ISTIMEWA YOGYAKARTA',
 'SUMATERA SELATAN',
 'KALIMANTAN TENGAH',
 'JAMBI',
 'RIAU',
 'KALIMANTAN BARAT',
 'BENGKULU',
 'KALIMANTAN TIMUR',
 'NUSA TENGGARA BARAT',
 'KEPULAUAN RIAU',
 'GORONTALO',
 'SULAWESI UTARA',
 'SULAWESI TENGGARA',
 'KALIMANTAN UTARA',
 'MALUKU',
 'PAPUA BARAT',
 'MALUKU UTARA',
 'DKI JAKARTA',
 'LAMPUNG',
 'BANTEN',
 'SULAWESI BARAT']

# FUNCTION 

In [152]:
def format_type(source):
    x = source.copy()
    colum = x.columns.to_list()
    new_colum = [item for item in colum if item not in ['% TARGET GP 2024','Kelurahan','No']]
    x['Kelurahan'] = x['Kelurahan'].astype('str')
    
    for i in new_colum:
        x[i] = x[i].astype('int64')
    
    total = x[['Jumlah TPS','DPT 2024','DPT 2024 x 82%','Target GP 2024']].sum().to_frame().T
    
    return [total, x]

def total(tipe, data):
    type_data = format_type(data) 
    type_data[0]['Kelurahan'] = tipe
    type_data[0]['Kuadran'] = 5
    
    df = pd.concat([type_data[1], type_data[0]]) 
    df = df.reset_index(drop=True)
    return df

def move_column(data):

    first_column = data.pop('Jumlah DPT 2024')
    two_column = data.pop('DPT 2024 x 82%')
    thre_column = data.pop('Target GP 2024')
    tps = data.pop('Jumlah TPS')
    target_gp_per = data.pop('% TARGET GP 2024')
    No = data.pop('angka')
    kec = data.pop('Kelurahan')
    
    data.insert(0, 'No', No)
    data.insert(1, 'Kelurahan', kec)
    data.insert(2, 'Jumlah TPS', tps)
    data.insert(3, 'DPT 2024', first_column)
    data.insert(4, 'DPT 2024 x 82%', two_column)
    data.insert(5, 'Target GP 2024', thre_column)
    data.insert(6, '% TARGET GP 2024', target_gp_per)

    data['No'] = data['No'].astype('str')
    
    return data

def format_color_groups(df):
    colors = ['','#FFCECD', '#FFF4D9','#FFCECD','#EBF1F1','#FFE7E7','green']
    x = df.copy()
    factors = list(x['Kuadran'].unique())

    for factor in factors:
        style = f'background-color: {colors[int(factor)]}; color:black;'
        x.loc[x['Kuadran'] == factor, :] = style
    
    return x

def table_style_quadran(source):
    filter = source.columns.to_list()
    format_dict = {col: '{:,}' for col in filter if col in ['DPT 2024', 'DPT 2024 x 82%', 'Target GP 2024']}

    for col in filter:
        
        font_css = '"Montserrat", sans-serif;'
        header_styles = {
            col : [{'selector': 'th',  'props': 'background-color: #db2016; color: white; text-align: center; font-weight: 700; font-family: "Montserrat", sans-serif;'}],
        }

        source = source.set_table_styles(header_styles, overwrite=False).format(format_dict, na_rep='').set_properties(**{'text-align':'center','font-family': font_css})

    return source

# def capture_micro_target(x, y, z):
#     prio = x
#     non_prio = y
#     prio_non_prio = z


In [ ]:
def format_color_groups(df):
    colors = ['','#FFCECD', '#FFF4D9','#FFCECD','#EBF1F1','#FFE7E7','green']
    x = df.copy()
    # factors = list(x['Kuadran'].unique())
    print(x['Kuadran'])
    # for factor in factors:
    #     if factor < len(colors):
    #         print(colors[int(factor)])

asw = non_prio.apply(format_color_groups)
asw

In [137]:
non_prio['Kuadran'].unique()

array([4, 8], dtype=int64)

In [148]:
non_prio

,No,Kelurahan,Jumlah TPS,DPT 2024,DPT 2024 x 82%,Target GP 2024,% TARGET GP 2024,Kuadran
0,1,BLANG RIKUI,2.0,559,458,7,1.60%,4
1,2,DAYA BAROH,2.0,385,316,8,2.50%,4
2,3,DAYA COT,2.0,440,361,7,2.00%,4
3,4,DAYA KAMPONG BARO,2.0,444,364,7,1.80%,4
4,5,DAYA TEUNGOH,2.0,315,258,7,2.70%,4
5,6,LHOK IGEUH,1.0,72,59,34,58.10%,4
6,7,MAMPREE,2.0,378,310,14,4.40%,4
7,8,MEUNASAH MANCANG,1.0,205,168,30,18.00%,4
8,9,MEUNASAH PANAH,2.0,409,335,18,5.30%,4
9,10,PANTON BEUNOT,1.0,90,74,32,43.90%,4


# TINGKAT KABUPATEN KECAMATAN KELURAHAN

In [153]:
def prioritas(prio):
    prio.reset_index(inplace=True)
    prio['angka'] = prio.index + 1
    move_column_prio = move_column(prio)
    total_prio = total('Total Prioritas', move_column_prio)
    color_prio = total_prio.style.hide().apply(format_color_groups, axis=None)
    style_prio = table_style_quadran(color_prio).set_table_styles(styles, overwrite=False).set_properties(subset=total_prio.index[-1],**{'font-weight':'700'}).hide(['Kuadran'], axis=1)

    return dfi.export(style_prio, f'{quadran_microtargeting}/KECAMATAN-{kec}-prioritas.jpg')

def non_prioritas(non_prio):
    non_prio.reset_index(inplace=True)
    non_prio['angka'] = non_prio.index + 1
    move_column_non_prio = move_column(non_prio)
    total_non_prio = total('Total Nonprioritas', move_column_non_prio)
    color_non_prio = total_non_prio.style.hide().apply(format_color_groups, axis=None)
    style_non_prio = table_style_quadran(color_non_prio).set_table_styles(styles, overwrite=False).set_properties(subset=total_non_prio.index[-1],**{'font-weight':'700'}).hide(['Kuadran'], axis=1)

    return dfi.export(style_non_prio, f'{quadran_microtargeting}/KECAMATAN-{kec}-non_prioritas.jpg')

def grand_total_prio_non_prio(prio_non_prio):
    prio_non_prio.reset_index(inplace=True)
    prio_non_prio['angka'] = prio_non_prio.index + 1
    move_column_prio_non_prio = move_column(prio_non_prio)
    grand_total = format_type(move_column_prio_non_prio)
    grand_total[0][''] = 'Total Prioritas + Nonprioritas'
    grand_total[0]['% TARGET GP 2024'] = ''
    grand_total[0]['Kuadran'] = 5

    kecamatan = grand_total[0].pop('')

    grand_total[0].insert(0, '', kecamatan)
    colors_grand_total= grand_total[0].style.hide().apply(format_color_groups, axis=None)
    styled_grand_total = table_style_quadran(colors_grand_total).set_properties(**{'font-weight':'700'}).hide(['Kuadran'], axis=1)

    return dfi.export(styled_grand_total, f'{quadran_microtargeting}/KECAMATAN-{kec}-grand_total.jpg')

In [154]:
# kecamatan = ['PENANGGALAN']
# kecamatan

styles = {
    'Kelurahan' : [{'selector': 'td', 'props': [('text-align', 'left')]}]
}

for prop in provinsi:
    for kab in kabkot:
        kecamatan_filtered = df_quadran[(df_quadran['PROPINSI'] == prop) & (df_quadran['KABUPATEN2'] == kab)]['Kecamatan'].unique()
        
        for kec in kecamatan_filtered:
            prio = df_quadran[(df_quadran['PROPINSI'] == prop) & (df_quadran['KABUPATEN2'] == kab) & (df_quadran['Kecamatan'] == kec) & (df_quadran['Prio'] == 1)].pivot_table(index='Kelurahan', values=colum, aggfunc='sum')
            non_prio = df_quadran[(df_quadran['PROPINSI'] == prop) & (df_quadran['KABUPATEN2'] == kab) & (df_quadran['Kecamatan'] == kec) & (df_quadran['Prio'] == 0)].pivot_table(index='Kelurahan', values=colum, aggfunc='sum')
            prio_non_prio = df_quadran[(df_quadran['PROPINSI'] == prop) & (df_quadran['KABUPATEN2'] == kab) & (df_quadran['Kecamatan'] == kec)].pivot_table(index='Kelurahan', values=colum, aggfunc='sum')       

            prio_is_empty = prio.empty
            non_prio_is_empty = non_prio.empty 
            quadran_microtargeting = f'../quadran/coba/{prop}/{kab}/tingkat_kecamatan'

            if not os.path.exists(quadran_microtargeting):
                os.makedirs(quadran_microtargeting)

            if prio_is_empty:
                if non_prio_is_empty:
                    #jika keduanya kosong
                    pass
                else:
                    #jika prio kosong export non prio dan grand total
                    result_non_prio = non_prioritas(non_prio)
                    result_grand_total = grand_total_prio_non_prio(prio_non_prio)
                    
            else:
                if non_prio_is_empty:
                    #jika non prio kosong exsport prio dan grand total
                    result_prio = prioritas(prio)
                    result_grand_total = grand_total_prio_non_prio(prio_non_prio)
                else:
                    #jika keduanya prio dan non prio tidak kosong export keduanya
                    result_prio = prioritas(prio)
                    result_non_prio = non_prioritas(non_prio)
                    result_grand_total = grand_total_prio_non_prio(prio_non_prio)

            # non_prio.reset_index(inplace=True)
            # prio_non_prio.reset_index(inplace=True)

            # prio['angka'] = prio.index + 1
            # non_prio['angka'] = non_prio.index + 1
            # prio_non_prio['angka'] = prio_non_prio.index + 1
            
            # move_column_prio = move_column(prio)
            # move_column_non_prio = move_column(non_prio)
            # move_column_prio_non_prio = move_column(prio_non_prio)

            # total_prio = total('Total Prioritas', move_column_prio)
            # total_non_prio = total('Total Nonprioritas', move_column_non_prio)

            # grand_total = format_type(move_column_prio_non_prio)
            # grand_total[0][''] = 'Total Prioritas + Nonprioritas'
            # grand_total[0]['% TARGET GP 2024'] = ''
            # grand_total[0]['Kuadran'] = 5

            # kecamatan = grand_total[0].pop('')

            # grand_total[0].insert(0, '', kecamatan)

            # color_prio = total_prio.style.hide().apply(format_color_groups, axis=None).hide(['Kuadran'], axis=1)
            # color_non_prio = total_non_prio.style.hide().apply(format_color_groups, axis=None).hide(['Kuadran'], axis=1)
            # colors_grand_total= grand_total[0].style.hide().apply(format_color_groups, axis=None).hide(['Kuadran'], axis=1)

            # style_prio = table_style_quadran(color_prio).set_table_styles(styles, overwrite=False).set_properties(subset=total_prio.index[-1],**{'font-weight':'700'})
            # style_non_prio = table_style_quadran(color_non_prio).set_table_styles(styles, overwrite=False).set_properties(subset=total_non_prio.index[-1],**{'font-weight':'700'})
            # styled_grand_total = table_style_quadran(colors_grand_total).set_properties(**{'font-weight':'700'})

C:\Users\User\AppData\Local\Temp\ipykernel_20864\2332678467.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'background-color: #EBF1F1; color:black;' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  x.loc[x['Kuadran'] == factor, :] = style
C:\Users\User\AppData\Local\Temp\ipykernel_20864\2332678467.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'background-color: #FFE7E7; color:black;' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  x.loc[x['Kuadran'] == factor, :] = style
C:\Users\User\AppData\Local\Temp\ipykernel_20864\2332678467.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'background-color: #EBF1F1; color:black;' has dtype incompatible with int64, please explicitly cast to a

In [94]:
result_prio

# CAPTURE PILEG AND PILPRES WITH COLOR

In [ ]:
# file_pilpres = '../request_mas_zima/capture_prioritas/0111014_2019_suara_pilpres_kel.csv'
# file_dpr_ri = '../request_mas_zima/capture_prioritas/0112024_2019_pers_suara_dpr_ri_kel.csv'

# pileg = pd.read_csv(file_dpr_ri)
# pilpres = pd.read_csv(file_pilpres)

# pilpres.head()

,nama_pro,nama_kab,nama_kec,Kelurahan,JW-MA,PS-SU,JW-MA(%),PS-SU(%)
0,ACEH,ACEH SELATAN,BAKONGAN,KEUDE BAKONGAN,92,611,0.130868,0.869132
1,ACEH,ACEH SELATAN,BAKONGAN,UJONG MANGKI,89,344,0.205543,0.794457
2,ACEH,ACEH SELATAN,BAKONGAN,UJONG PADANG,29,362,0.074169,0.925831
3,ACEH,ACEH SELATAN,BAKONGAN,GAMPONG DRIEN,44,222,0.165414,0.834586
4,ACEH,ACEH SELATAN,KLUET UTARA,FAJAR HARAPAN,35,371,0.086207,0.913793


In [ ]:
pileg.head()

,nama_pro,nama_kab,nama_kec,Kelurahan,PKB,Gerindra,PDIP,Golkar,Nasdem,Garuda,Berkarya,PKS,Perindo,PPP,PSI,PAN,Hanura,Demokrat,PA,Partai SIRA,PD Aceh,PNA,PBB,PKPI
0,ACEH,ACEH SELATAN,BAKONGAN,KEUDE BAKONGAN,0.013081,0.011628,0.002907,0.082849,0.010174,0.0,0.005814,0.014535,0.005814,0.728198,0.005814,0.015988,0.005814,0.085756,0.0,0.0,0.0,0.0,0.001453,0.010174
1,ACEH,ACEH SELATAN,BAKONGAN,UJONG MANGKI,0.016908,0.024155,0.004831,0.108696,0.016908,0.0,0.012077,0.012077,0.009662,0.521739,0.007246,0.060386,0.014493,0.140097,0.0,0.0,0.0,0.0,0.026570,0.014493
2,ACEH,ACEH SELATAN,BAKONGAN,UJONG PADANG,0.021277,0.026596,0.000000,0.023936,0.007979,0.0,0.007979,0.029255,0.000000,0.699468,0.007979,0.055851,0.000000,0.082447,0.0,0.0,0.0,0.0,0.010638,0.018617
3,ACEH,ACEH SELATAN,BAKONGAN,GAMPONG DRIEN,0.020243,0.040486,0.000000,0.020243,0.004049,0.0,0.016194,0.012146,0.000000,0.797571,0.004049,0.016194,0.008097,0.044534,0.0,0.0,0.0,0.0,0.004049,0.008097
4,ACEH,ACEH SELATAN,KLUET UTARA,FAJAR HARAPAN,0.043814,0.048969,0.005155,0.141753,0.077320,0.0,0.015464,0.025773,0.005155,0.118557,0.025773,0.337629,0.015464,0.079897,0.0,0.0,0.0,0.0,0.036082,0.018041


In [ ]:
columns_pileg = pileg.columns.to_list()
columns_pilpres = pilpres.columns.to_list()
provinsi = pileg['nama_pro'].unique().tolist()
kabupaten = pileg['nama_kab'].unique().tolist()
kecamatan = pileg['nama_kec'].unique().tolist()

#provinsi = ['BANTEN']
columns_pilpres

['nama_pro',
 'nama_kab',
 'nama_kec',
 'Kelurahan',
 'JW-MA',
 'PS-SU',
 'JW-MA(%)',
 'PS-SU(%)']

In [ ]:
#pileg['Max Value'] = pileg[columns_pileg[7:]].max(axis=1)
pileg['pemenang'] = pileg[columns_pileg[4:]].idxmax(axis=1)

pilpres['Max Value'] = pilpres[columns_pilpres[6:]].max(axis=1)
pilpres['pemenang'] = pilpres[columns_pilpres[6:]].idxmax(axis=1)

columns_pileg = pileg.columns.to_list()

pilpres.head(5)

,nama_pro,nama_kab,nama_kec,Kelurahan,JW-MA,PS-SU,JW-MA(%),PS-SU(%),Max Value,pemenang
0,ACEH,ACEH SELATAN,BAKONGAN,KEUDE BAKONGAN,92,611,0.130868,0.869132,0.869132,PS-SU(%)
1,ACEH,ACEH SELATAN,BAKONGAN,UJONG MANGKI,89,344,0.205543,0.794457,0.794457,PS-SU(%)
2,ACEH,ACEH SELATAN,BAKONGAN,UJONG PADANG,29,362,0.074169,0.925831,0.925831,PS-SU(%)
3,ACEH,ACEH SELATAN,BAKONGAN,GAMPONG DRIEN,44,222,0.165414,0.834586,0.834586,PS-SU(%)
4,ACEH,ACEH SELATAN,KLUET UTARA,FAJAR HARAPAN,35,371,0.086207,0.913793,0.913793,PS-SU(%)


# FUNCTION

In [ ]:
warna_partai = {
    'PDIP': 'background-color: #db2016; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Kabupaten/Kota': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'Kota / Kab': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'Kecamatan': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'Kelurahan': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'JW-MA': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'PS-SU': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'JW-MA(%)': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'PS-SU(%)': 'background-color: #db2016; color: white; font-weight: 700; text-align: center; font-family: "Montserrat", sans-serif;',
    'Golkar': 'background-color: #ffff00 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Gerindra': 'background-color: #b79164 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Nasdem': 'background-color: #99d9ea ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKB': 'background-color: #008000 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Demokrat': 'background-color: #2643a3 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PAN': 'background-color: #0033ff ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKS': 'background-color: #000000 ; color: white; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PPP': 'background-color: #00b300 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Perindo': 'background-color: #7583a9 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Berkarya': 'background-color: #eeee22 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Hanura': 'background-color: #ed9a11 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PSI': 'background-color: #ff00ff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PBB': 'background-color: #00ff00 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Garuda': 'background-color: #e6b333 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PKPI': 'background-color: #ff0000 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PD Aceh': 'background-color: #118dff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PA': 'background-color: #fa8072 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'Partai SIRA': 'background-color: #00beff ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'PNA': 'background-color: #fbceb1 ; color: black; font-weight: 700; font-family: "Montserrat", sans-serif;',
    'pemenang':'',
    'pilpres_jwma':''}

def select_col(source):
    
    filter = source.columns.to_list()
    filter.remove('Max Value')

    df1 =  pd.DataFrame(index=source.index, columns=source.columns)

    for col in filter[1:]:
        compare = source[col].eq(source['Max Value'])
        df1.loc[compare, col] = warna_partai[col]

    return df1

def table_style(source):
    filter = source.columns.to_list()
    filter.remove('Max Value')

    for col in filter:

        font_css = '"Montserrat", sans-serif;'
        header_styles = {
            col : [{'selector': 'th',  'props': warna_partai[col]}],
        }
        
        source = source.set_table_styles(header_styles, overwrite=False)
        
        format_dict = {col: '{:.1%}' for col in filter[1:]}
        convert = source.format(format_dict)
        
        convert.set_properties(**{'text-align':'center','font-family': font_css})

    return convert

def table_style_pilpres(source):
    filter = source.columns.to_list()
    filter.remove('Max Value')

    for col in filter:
        header_styles = {
            col : [{'selector': 'th',  'props': warna_partai[col]}],
        }
        
        source = source.set_table_styles(header_styles, overwrite=False)
        source.set_properties(**{'text-align':'center','font-family': '"Montserrat", sans-serif;'})

    return source

def move_column_pilpres(data):
    first_column = data.pop('PS-SU')
    two_column = data.pop('PS-SU(%)')	
    
    data.insert(2, 'PS-SU', first_column)
    data.insert(4, 'PS-SU(%)', two_column)

    return data

def apply_style(row, move_pilpres):
    background = {
        'A' : 'background-color: #eaa00a; color: white; font-weight: bold;',
        'B' : 'background-color: #ffba14; color: white; font-weight: bold;',
        'C' : 'background-color: #fed12b; color: white; font-weight: bold;', 
        'D' : 'background-color: #f9d97d; color: white; font-weight: bold;', 
        'F' : 'background-color: #ff8080; color: white; font-weight: bold;', 
        'G' : 'background-color: #ff6666; color: white; font-weight: bold;', 
        'H' : 'background-color: #ff5550; color: white; font-weight: bold;',
        'I' : 'background-color: #ff2929; color: white; font-weight: bold;'}

    styles = {col: '' for col in move_pilpres.columns}
    
    for col in move_pilpres.columns:
        if row['pemenang'] == col and col in ['JW-MA(%)', 'PS-SU(%)']:
            styles[col] = background[row['pilpres_jwma']]
    return pd.Series(styles, index=move_pilpres.columns)  

def classify_pilpres(jwma):
    if 0 <= jwma < 0.15:
        return 'A'
    elif 0.15 <= jwma < 0.30:
        return 'B'
    elif 0.30 <= jwma < 0.45:
        return 'C'
    elif 0.45 <= jwma < 0.50:
        return 'D'
    elif 0.50 <= jwma < 0.55:
        return 'F'
    elif 0.55 <= jwma < 0.70:
        return 'G'
    elif 0.70 <= jwma < 0.85:
        return 'H'
    else:
        return 'I'
    

styles = {
    'Kelurahan' : [{'selector': 'td', 'props': [('text-align', 'left')]}]
}

tes = []
pilpres['pilpres_jwma'] = pilpres['JW-MA(%)'].apply(classify_pilpres)
columns_pilpres = pilpres.columns.to_list()

def capture_pileg_pilpres(unpivot_pileg, unpivot_pilpres, tingkat):

    if unpivot_pileg.empty or unpivot_pilpres.empty:
        pass
    else:
        #pilpres
        unpivot_pilpres.reset_index(inplace=True)
        move_pilpres = move_column_pilpres(unpivot_pilpres)
        color_pilpres = move_pilpres.style.hide().apply(apply_style, move_pilpres=move_pilpres, axis=1)
        hide_col = color_pilpres.hide(['Max Value','pemenang','pilpres_jwma'], axis=1)
        style_pilpres = table_style_pilpres(hide_col)
        format_dict = {'JW-MA(%)': '{:.1%}', 'PS-SU(%)': '{:.1%}', 'JW-MA':'{:,}', 'PS-SU':'{:,}'}
        convert_pilpres = style_pilpres.format(format_dict)
        convert_pilpres = convert_pilpres.set_table_styles(styles, overwrite=False)

        #pileg
        unpivot_pileg.reset_index(inplace=True)
        unpivot_pileg.index = unpivot_pileg.index + 1
        pemenang_pileg = unpivot_pileg['pemenang'].unique().tolist()
        pemenang_pileg.insert(0, tingkat)
        filter_pileg = unpivot_pileg[pemenang_pileg]
        filter_pileg['Max Value'] = unpivot_pileg[pemenang_pileg[1:]].max(axis=1)

        coloring_pileg = filter_pileg.style.hide().apply(select_col, axis=None)
        convert_pileg = coloring_pileg.set_table_styles(styles, overwrite=False)
        update_pileg = convert_pileg.hide(['Max Value'], axis=1)
        update_pileg = table_style(update_pileg)
        
        presiden = f'../request_mas_zima/capture_prioritas/tes/presiden/{pro}/{kab}/tingkat_kecamatan/'
        dpr_ri = f'../request_mas_zima/capture_prioritas/tes/dpr_ri/{pro}/{kab}/tingkat_kecamatan/'

        if not os.path.exists(presiden) and not os.path.exists(dpr_ri):
            os.makedirs(presiden)
            os.makedirs(dpr_ri)

        dfi.export(update_pileg, f'{dpr_ri}/KECAMATAN-{kec}.jpg')
        dfi.export(convert_pilpres, f'{presiden}/KECAMATAN-{kec}.jpg')

# TINGKAT KABUPATEN KECAMATAN KELURAHAN

In [ ]:
for pro in provinsi:
    for kab in kabupaten:
        for kec in kecamatan:
            unpivot_pileg = pileg[(pileg['nama_pro'] == pro) & (pileg['nama_kab'] == kab) & (pileg['nama_kec'] == kec)].pivot_table(index='Kelurahan', values=columns_pileg[4:], aggfunc='sum')
            unpivot_pilpres = pilpres[(pilpres['nama_pro'] == pro) & (pilpres['nama_kab'] == kab) & (pileg['nama_kec'] == kec)].pivot_table(index='Kelurahan', values=columns_pilpres[4:], aggfunc='sum')

            capture_pileg_pilpres(unpivot_pileg, unpivot_pilpres, 'Kelurahan')

C:\Users\User\AppData\Local\Temp\ipykernel_18788\1147780324.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_pileg['Max Value'] = unpivot_pileg[pemenang_pileg[1:]].max(axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_18788\1147780324.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_pileg['Max Value'] = unpivot_pileg[pemenang_pileg[1:]].max(axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_18788\1147780324.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

KeyboardInterrupt: 

In [ ]:
columns_pilpres[3:]

['Kelurahan', 'JW-MA', 'PS-SU', 'JW-MA(%)', 'PS-SU(%)']

# COBA

In [ ]:
styles = {
    'Kecamatan' : [{'selector': 'td', 'props': [('text-align', 'left')]}]
}
tes = []
pilpres['pilpres_jwma'] = pilpres['JW-MA(%)'].apply(classify_pilpres)
columns_pilpres = pilpres.columns.to_list()

for pro in provinsi:
    for kab in kabupaten:
        unpivot_pileg = pileg[(pileg['nama_pro'] == pro) & (pileg['nama_kab'] == kab)].pivot_table(index='Kecamatan', values=columns_pileg[7:], aggfunc='sum')
        unpivot_pilpres = pilpres[(pilpres['nama_pro'] == pro) & (pilpres['nama_kab'] == kab)].pivot_table(index='Kecamatan', values=columns_pilpres[7:], aggfunc='sum')

        if unpivot_pileg.empty or unpivot_pilpres.empty:
            pass
        else:
            #pilpres
            unpivot_pilpres.reset_index(inplace=True)
            move_pilpres = move_column_pilpres(unpivot_pilpres)
            color_pilpres = move_pilpres.style.hide().apply(apply_style, axis=1)
            hide_col = color_pilpres.hide(['Max Value','pemenang','pilpres_jwma'], axis=1)
            style_pilpres = table_style_pilpres(hide_col)
            format_dict = {'JW-MA(%)': '{:.1%}', 'PS-SU(%)': '{:.1%}', 'JW-MA':'{:,}', 'PS-SU':'{:,}'}
            convert_pilpres = style_pilpres.format(format_dict)
            convert_pilpres = convert_pilpres.set_table_styles(styles, overwrite=False)
            #coloring_pileg = move_pilpres.style.apply(table_style, axis=None)

            #pileg
            # unpivot_pileg.reset_index(inplace=True)
            # unpivot_pileg.index = unpivot_pileg.index + 1
            # pemenang_pileg = unpivot_pileg['pemenang'].unique().tolist()
            # pemenang_pileg.insert(0, 'Kecamatan')
            # filter_pileg = unpivot_pileg[pemenang_pileg]
            # filter_pileg['Max Value'] = unpivot_pileg[pemenang_pileg[1:]].max(axis=1)

            # coloring_pileg = filter_pileg.style.hide().apply(select_col, axis=None)
            # convert_pileg = coloring_pileg.set_table_styles(styles, overwrite=False)
            # update_pileg = convert_pileg.hide(['Max Value'], axis=1)
            # update_pileg = table_style(update_pileg)
            
            presiden = f'../request_mas_zima/capture_prioritas/tes/{pro}'
            dpr_ri = f'../request_mas_zima/capture_prioritas/capture_dpr_ri/{pro}'

            if not os.path.exists(presiden):
                os.makedirs(presiden)

            #dfi.export(update_pileg, f'{dpr_ri}/KABUPATEN-{kab}.jpg')
            dfi.export(convert_pilpres, f'{presiden}/KABUPATEN-{kab}.jpg')